In [14]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

from utils.auxfunctions import moving_window
import numpy as np
import matplotlib.pyplot as plt

In [15]:
hidden_dim = 6
n_signals = 1
N = 64
tagset_size = 1

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [16]:
class LSTM_freq(nn.Module):
    """Some Information about LSTM_freq"""
    def __init__(self, n_signals, hidden_dim, tagset_size):
        super(LSTM_freq, self).__init__()
        self.lstm = nn.LSTM(n_signals, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, tagset_size)
        

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        print(lstm_out.shape)
        fc_layer = self.fc(lstm_out[:,-1,:])
        print(fc_layer.shape)
        # prob_tag = self.activation(fc_layer)
        # print(prob_tag.shape)

        return torch.sigmoid(fc_layer


In [17]:
model = LSTM_freq(n_signals, hidden_dim, tagset_size).to(device)
loss_fn = nn.CrossEntropyLoss()
# loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
print(signals[0:2, :].shape)
x = model(signals[0:2, :].to(device))
print(x)


torch.Size([2, 64, 1])


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [7]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:]} \n")

Model structure: LSTM_freq(
  (lstm): LSTM(1, 6, batch_first=True)
  (fc): Linear(in_features=6, out_features=1, bias=True)
  (activation): Sigmoid()
)


Layer: lstm.weight_ih_l0 | Size: torch.Size([24, 1]) | Values : tensor([[ 0.0964],
        [ 0.2240],
        [ 0.4032],
        [ 0.1122],
        [ 0.0065],
        [-0.2128],
        [-0.0526],
        [-0.3949],
        [ 0.2505],
        [-0.2095],
        [-0.3706],
        [-0.0529],
        [-0.3886],
        [ 0.1614],
        [-0.3385],
        [ 0.0736],
        [-0.2268],
        [-0.0452],
        [ 0.1280],
        [-0.3813],
        [ 0.1443],
        [ 0.2201],
        [-0.1129],
        [ 0.2981]], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: lstm.weight_hh_l0 | Size: torch.Size([24, 6]) | Values : tensor([[-0.2922, -0.2111, -0.0093,  0.3135,  0.2678, -0.1862],
        [ 0.1262,  0.1472,  0.0480, -0.3947, -0.2282,  0.3718],
        [ 0.2747, -0.2474,  0.2735,  0.2223,  0.3949, -0.2846],
        [ 0.1957, -0.314

In [8]:
class SineWaveDataset(torch.utils.data.Dataset):
    """Some Information about SineWaveDataset
    
    signals: dim -> (m, N)

    """

    def __init__(self, signals, labels, transform=None, target_transform=None):
        super(SineWaveDataset, self).__init__()
        self.signals = signals
        self.labels = labels
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.signals)

    def __getitem__(self, index):
        signal = self.signals[index]
        label = self.labels[index]
        if self.transform:
            signal = self.transform(signal)
        if self.target_transform:
            label = self.target_transform(label)
        # return torch.tensor(signal).float(), label
        return signal, label
    
class MockDataset(torch.utils.data.Dataset):
    """Some Information about MockDataset"""
    def __init__(self, signals):
        super(MockDataset, self).__init__()
        self.signals = signals

    def __getitem__(self, index):
        return self.signals[index]
        
    def __len__(self):
        return len(self.signals)


In [5]:
fs = 3840
N = 64
# Create Dataset


t = np.linspace(0,N/fs,N)
m = 300
signals = t
target = np.array([[0]])

for i in (np.linspace(0, 10, int(m/2))):
    signal = np.array(i*np.sin(2*np.pi*fs*t))
    signals = np.vstack((signals, signal))
    target = np.vstack((target, np.array(1)))
signals = signals[:-1]
target = target[:-1]
target
for i in range(int(m/4)):
    signal = np.random.normal(scale=3, size=N)
    signals = np.vstack((signals, signal))
    target = np.vstack((target, np.array(0)))
    
for i in range(int(m/4)):
    signal = np.ones(64)*i/10
    signals = np.vstack((signals, signal))
    target = np.vstack((target, np.array(0)))

signals = np.expand_dims(signals, axis=2)
signals = torch.from_numpy(signals).float()
target = torch.from_numpy(target).float()
print(target.shape)
print(signals.shape)


torch.Size([300, 1])
torch.Size([300, 64, 1])


In [9]:
dataset = SineWaveDataset(signals, target)
train_dataset, test_dataset, dev_dataset  = random_split(dataset, [0.9, 0.05, 0.05])
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=True)
dev_dataloader = DataLoader(dev_dataset, batch_size=16, shuffle=True)






In [10]:
# Training
def train(dataloader, model, loss_fn, optimizer):
    size=len(dataloader.dataset)
    model.train()
    for batch, (x, y) in enumerate(dataloader):
        x, y = x.to(device), y.to(device)

        # Compute prediction error
        pred = model(x)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 30 == 0:
            loss, current = loss.item(), batch * len(x)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")




In [11]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            pred = model(x)
            prob = torch.sigmoid(pred)
            test_loss += loss_fn(pred, y).item()
            correct += (torch.round(prob) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [12]:
CUDA_LAUNCH_BLOCKING =1
epochs = 8
for t in range(epochs):
    print(f"Epoch {t+1}\n-----------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-----------------------------------
torch.Size([16, 64, 6])
torch.Size([16, 1])
loss: 0.451900  [    0/  270]
torch.Size([16, 64, 6])
torch.Size([16, 1])


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [12]:
test_loss, correct = 0, 0
for batch, (x, y) in enumerate(dev_dataloader):
    x, y = x.to(device), y.to(device)
    pred = model(x)
    prob = torch.sigmoid(pred)
    loss = loss_fn(pred, y)
    correct += (torch.round(prob) == y).type(torch.float).sum().item()
    print(pred)
    print(prob)
    print(y)
    
    
    

print(correct)
print(loss)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:

dataiter = iter(test_dataloader)
signal, labels = next(dataiter)


print(labels[1:10])
print(signal.shape)
figure = plt.figure(figsize=(10,10))
cols, rows = 3, 3
for i in range(1, cols * rows +1):
    figure.add_subplot(rows, cols, i)
    plt.plot(t, signal[i,:])


In [11]:
CUDA_LAUNCH_BLOCKING =1
test_loss, correct = 0, 0
for batch, (x, y) in enumerate(train_dataloader):
    print(f"Batch: {batch}, label: {y[0]}")
    x, y = x.to(device), y.to(device)
    pred = model(x)
    prob = torch.sigmoid(pred)
    loss = loss_fn(pred, y)
    correct += (torch.round(prob) == y).type(torch.float).sum().item()
    print(pred)
    
    print(y)
    
    
    break
print(correct)
print(loss)
# print(prob)

Batch: 0, label: tensor([1.])


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.